If you run this in Google Colaboraotry, install and import the required libraries. Google Colab notebooks come with many libraries preinstalled, however there are some libraries that need to be installed manually.

For text preprocessing and cleaning, I use the spacy library.

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [1]:
# !pip install simpletransformers

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English
import string
import re
from bs4 import BeautifulSoup
from simpletransformers.classification import ClassificationModel 

/usr/local/lib/python3.7/site-packages/pandas/compat/__init__.py:120: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)
I0827 18:41:21.429533 139812794814592 file_utils.py:39] PyTorch version 1.1.0 available.
I0827 18:41:22.534654 139812794814592 file_utils.py:55] TensorFlow version 2.1.0 available.
wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.
I0827 18:41:22.819866 139812794814592 _import_c_extension.py:31] Failed to import cuda module: No module named 'caffe2.python.caffe2_pybind11_state_gpu'
I0827 18:41:22.820994 139812794814592 _import_c_extension.py:38] Failed to import AMD hip module: No module named 'caffe2.python.caffe2_pybind11_state_hip'
W0827 18:41:22.821453 139812794814592 _import_c_extension.py:41] This caffe2 python run does not have GPU support. Will run in CPU only mode.


# Basic data cleaning and data exploration before cleaning.

In [3]:
#clean data
punctuations = [',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';', "'", '$', '&', '/', '[', ']', '>', '%', '=', '#', '*', '+', '\\', '•',  '~', '@', '£',
 '·', '_', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',  '♥', '←', '×', '§', '″', '′', 'Â', '█', '½', 'à', '…', '\xa0', '\t',
 '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', '\u3000', '\u202f',
 '▒', '：', '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', '«',
 '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√', ]

expansion_dict = {"aren't" : "are not",
"can't" : "cannot",
"couldn't" : "could not",
"couldnt" : "could not",
"didn't" : "did not",
"doesn't" : "does not",
"doesnt" : "does not",
"don't" : "do not",
"hadn't" : "had not",
"hasn't" : "has not",
"haven't" : "have not",
"havent" : "have not",
"he'd" : "he would",
"he'll" : "he will",
"he's" : "he is",
"i'd" : "I would",
"i'd" : "I had",
"i'll" : "I will",
"i'm" : "I am",
"isn't" : "is not",
"it's" : "it is",
"it'll":"it will",
"i've" : "I have",
"let's" : "let us",
"mightn't" : "might not",
"mustn't" : "must not",
"shan't" : "shall not",
"she'd" : "she would",
"she'll" : "she will",
"she's" : "she is",
"shouldn't" : "should not",
"shouldnt" : "should not",
"that's" : "that is",
"thats" : "that is",
"there's" : "there is",
"theres" : "there is",
"they'd" : "they would",
"they'll" : "they will",
"they're" : "they are",
"theyre":  "they are",
"they've" : "they have",
"we'd" : "we would",
"we're" : "we are",
"weren't" : "were not",
"we've" : "we have",
"what'll" : "what will",
"what're" : "what are",
"what's" : "what is",
"what've" : "what have",
"where's" : "where is",
"who'd" : "who would",
"who'll" : "who will",
"who're" : "who are",
"who's" : "who is",
"who've" : "who have",
"won't" : "will not",
"wouldn't" : "would not",
"you'd" : "you would",
"you'll" : "you will",
"you're" : "you are",
"you've" : "you have",
"'re": " are",
"wasn't": "was not",
"we'll":" will",
"didn't": "did not"}

punctuations = punctuations + list(string.punctuation)
stopwords = list(STOP_WORDS)
parser = English()

In [4]:
def spacy_tokenizer(sentence):
    tokens = parser(sentence)
    tokens = [word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in tokens]
    tokens = [word for word in tokens if word not in stopwords and word not in punctuations]
    tokens = " ".join([i for i in tokens])
    return tokens

def clean_text(x):
    x = str(x).replace("\n","")
    for punct in punctuations:
        x = x.replace(punct, "")
    return x

def clean_numbers(x):
    x = re.sub('\d+', ' ', x)
    return x

def expand(text):
    mispellings_re = re.compile('(%s)' % '|'.join(expansion_dict.keys()))

    def replace(match):
        return expansion_dict[match.group(0)]

    return mispellings_re.sub(replace, text)

def remove_space(string):
    string = BeautifulSoup(string).text.strip().lower()
    string = re.sub(r'((http)\S+)', 'http', string)
    string = re.sub(r'\s+', ' ', string)
    return string

def removeurl(raw_text):
    clean_text = re.sub(r'^https?:\/\/.*[\r\n]*', '', raw_text, flags=re.MULTILINE)
    return clean_text

In [5]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [6]:
#removing url tags
train['text'] = train['text'].apply(lambda x:removeurl(x))
test['text'] = test['text'].apply(lambda x:removeurl(x))

train['text'] = train['text'].apply(lambda x:clean_text(x))
test['text'] = test['text'].apply(lambda x:clean_text(x))

# train['text'] = train['text'].apply(lambda x:clean_numbers(x))
# test['text'] = test['text'].apply(lambda x:clean_numbers(x))

train['text'] = train['text'].apply(lambda x:expand(x))
test['text'] = test['text'].apply(lambda x:expand(x))

train['text'] = train['text'].apply(lambda x:remove_space(x))
test['text'] = test['text'].apply(lambda x:remove_space(x))

# data cleaning - The best result is obtained without applying spacy_tokenizer
# train['text'] = train['text'].apply(lambda x:spacy_tokenizer(x))
# test['text'] = test['text'].apply(lambda x:spacy_tokenizer(x))

In [7]:
def calculate_nr_words(text):
    return len(text.split(" "))
    
print(max(train['text'].apply(calculate_nr_words)))
print(max(test['text'].apply(calculate_nr_words)))

31
31


# Model training - Using Simple Transformers library.


In [8]:
X_train,X_val,Y_train,Y_val = train_test_split(train["text"],train["target"],test_size=0.2,random_state=10)

In [9]:
train_df = pd.concat([X_train, Y_train], ignore_index=True,axis=1)
train_df.columns = ["text", "label"]
print(train_df)
val_df = pd.concat([X_val, Y_val], ignore_index=True, axis=1)
val_df.columns = ["text", "label"]
print(val_df)

                                                   text  label
2572  msmigot wow what convincing amp compelling evi...      0
1813                                                         1
2767  70 years after atomic bombs japan still strugg...      1
6248  pyrbliss ah i remember those days in a snowsto...      1
7563                          i wrecked my stomach help      0
...                                                 ...    ...
3441  the dress memes have officially exploded on th...      1
1344  destruction magics fine just dont go burning d...      1
4623  incident with injuryi495 inner loop exit 31 md...      1
7293  win lisowski whitewashes the whirlwind 50 in r...      1
1289  mad river complex fires have burned 14028 acre...      1

[6090 rows x 2 columns]
                                                   text  label
6524  ddnewslive nitishkumar and arvindkejriwal cant...      0
701                acousticmaloley no he was blazing it      0
3119  do babies actually get e

In [11]:
args={'fp16': False,
      'num_train_epochs': 4,
      'overwrite_output_dir': True}

# Create a TransformerModel
model = ClassificationModel('roberta', 'distilroberta-base', args=args)

# Train the model
model.train_model(train_df, 
                            acc = accuracy_score, 
                            precision = precision_score,
                            recall = recall_score,
                            f1 = f1_score)

# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(val_df, 
                            acc = accuracy_score, 
                            precision = precision_score,
                            recall = recall_score,
                            f1 = f1_score)

In [ ]:
print(result)
accuracy = (result['tp'] + result['tn']) / (result['tp'] + result['tn'] + result['fp'] + result['fn'])
precision = (result['tp'] / (result['tp'] + result['fp']))
recall = (result['tp'] / (result['tp'] + result['fn']))
f1 = 2 * precision * recall / (precision + recall)
print("accuracy:" + str(accuracy))
print("precision:" + str(precision))
print("recall:" + str(recall))
print("f1:" + str(f1))

{'mcc': 0.6240092813522351, 'tp': 492, 'tn': 751, 'fp': 110, 'fn': 170, 'acc': 0.8161523309258043, 'precision': 0.8172757475083057, 'recall': 0.743202416918429, 'f1': 0.7784810126582279, 'eval_loss': 0.7195083945855518}
accuracy:0.8161523309258043
precision:0.8172757475083057
recall:0.743202416918429
f1:0.7784810126582279


In [ ]:
print(model_outputs)

[[ 1.8713876 -2.1478965]
 [ 2.0253356 -2.3642147]
 [ 0.5535431 -1.0907804]
 ...
 [-3.221645   3.1091406]
 [-3.338584   3.2489102]
 [ 2.4308846 -2.6668384]]


In [ ]:
predictions, raw_outputs = model.predict(test["text"]) 

This code is used to create a submission file for the Kaggle competition.

In [ ]:
submission_df = pd.concat([test["id"],pd.Series(predictions, name="target")], axis=1)
submission_df.head()
submission_df.to_csv("submission.csv", index=False)